In [104]:
import copy
import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import importlib
import life.lib
import life.lib as lib
importlib.reload(lib)
LOG = lib.simple_log.LOG
import math
from matplotlib import pyplot as plt
%matplotlib
plt.ion()
lib.utils.set_seed()
print(torch.__version__)
np.set_printoptions(precision=3)
%precision 4



Using matplotlib backend: TkAgg
2.0.0+cu117


'%.4f'

In [105]:
if "train_x" not in globals():
    x, y = lib.datasets.get_mnist_train()
    train_y = y % 2
    train_x = torch.reshape(x, (len(x), -1))
    x, y = lib.datasets.get_mnist_test()
    test_y = y % 2
    test_x = torch.reshape(x, (len(x), -1))

b = train_x[:, 0].numpy()



In [106]:
lib.utils.set_seed()
# network = lib.nets.PYRAMID()
#network = lib.nets.create_livenet_odd_2()
# train_x, train_y = lib.datasets.get_odd()
# network = lib.livenet.LiveNet(train_x.shape[1], None, torch.max(train_y) + 1)
network = lib.nets.PERCEPTRON(train_x.shape[1], torch.max(train_y) + 1, l1=0.1)
criterion = lib.nets.criterion_n

def create_optimizer(net: nn.Module):
    if net.__class__.__name__ == "LiveNet":
        print("LiveNet")
        net: lib.livenet.LiveNet
        optimizer = lib.livenet.LiveNetOptimizer(net, decay=0.0)
        # optimizer = torch.optim.Adam(net.parameters())
    else:
        print("Torch")
        optimizer = torch.optim.Adam(net.parameters(), betas=(0.0, 0.95))
        # optimizer = torch.optim.Adam(net.parameters())
    return optimizer

optimizer = create_optimizer(network)


Torch


In [107]:
params = [p for p in network.named_parameters()]
logits = network.forward(train_x)
for name, p in params:
    print(f"{name} {p.detach().numpy()}")
pred = network.forward(train_x)
loss = criterion(pred, train_y)
print(loss)
pred.detach().numpy()
# torch.onnx.export(network, train_x, "/home/spometun/model.onnx", verbose=False)




linear1.weight [[-0.     0.019 -0.029 ...  0.022  0.004  0.002]
 [-0.02  -0.015 -0.01  ... -0.02  -0.006 -0.03 ]]
linear1.bias [-0.02   0.015]
tensor(1.0976, grad_fn=<DivBackward0>)


array([[-0.019,  0.095],
       [-0.075,  0.143],
       [-0.086,  1.06 ],
       ...,
       [ 0.613,  0.59 ],
       [ 0.102,  0.75 ],
       [ 0.193,  0.402]], dtype=float32)

In [108]:
params_history = []

def get_gradients_dict(network: torch.nn.Module):
    res = {}
    for name, param in network.named_parameters():
        with torch.no_grad():
            res[name] = param.grad
    return res

g = get_gradients_dict(network)


In [136]:
optimizer.param_groups[0]["lr"] = 0.0001
network.alpha_l1 = 0.1
batch_size = 1000

n_epochs = 8
for i in range(n_epochs):
    e_counter = 0
    for start, end in lib.gen_utils.index_batcher(batch_size, len(train_x)):
        y = train_y[start:end]
        pred = network.forward(x)
        loss = criterion(pred, y)
        reg_loss = network.internal_loss()
        all_loss = loss + reg_loss
        optimizer.zero_grad()
        all_loss.backward()

        params = lib.utils.get_parameters_dict(network)
        grads = lib.utils.get_gradients_dict(network)
        params_history.append({"params": params, "grads": grads})

        optimizer.step()
        if e_counter == 0:
            LOG(f"{all_loss.detach().numpy():.3f} = {loss.detach().numpy():.3f}+{reg_loss.detach().numpy():.3f}")
        e_counter += 1



Iˈ0.000 0.886 = 0.701+0.186 
Iˈ0.065 0.886 = 0.700+0.186 
Iˈ0.123 0.886 = 0.699+0.187 
Iˈ0.180 0.886 = 0.699+0.187 
Iˈ0.236 0.886 = 0.699+0.187 
Iˈ0.296 0.886 = 0.697+0.189 
Iˈ0.352 0.886 = 0.697+0.188 
Iˈ0.415 0.886 = 0.696+0.190 


In [137]:

params_orig = [p.detach().clone().numpy() for p in network.parameters()]


In [138]:
# params = copy.deepcopy(params_orig)
#lib.utils.add_noise_to_params(params, 1, .0)
# lib.utils.set_parameters(network, params)


In [141]:
pred = network(train_x)
pred_bin = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = train_y - pred_bin
len(diff[diff != 0]) / len(diff)

0.1886

In [140]:
importlib.reload(lib)
%matplotlib
plt.ion()

def param_picker0(param):
    val0 = param["params"]["linear1.weight"][0][0].item()
    val1 = param["params"]["linear1.weight"][1][0].item()
    return val1 + val0

def param_picker1(param):
    val0 = param["params"]["linear1.weight"][0][1].item()
    return val0
    return param["grads"]["linear1.weight"][0][0].item() / 1000
    # return np.max(np.abs(param[0].numpy()))

def get_param_values(history, picker):
    values = []
    for entry in history:
        values.append(picker(entry))
    return values

plt.figure(figsize=(16, 9))
values = get_param_values(params_history, param_picker0)
# plt.plot(values)
values = get_param_values(params_history, param_picker1)
plt.plot(values)
plt.grid()

accum = lib.stat_utils.AccumStat()
accum.add_value(network.parameters())
#accum.add_value(params_history[-1])
accum.plot()




Using matplotlib backend: TkAgg


In [39]:
def form_balanced(x, y, n):
    assert n % 2 == 0
    n //= 2
    y = y.numpy()
    x = x.numpy()
    inds = np.argsort(y.squeeze(1), axis=0)
    y = y[inds]
    x = x[inds]
    i = 0
    batches_x = []
    batches_y = []
    while True:
        a = y[i:i + n]
        b = y[len(y) - i - n: len(y) - i]
        if not (a==0).all() or not (b==1).all():
            break
        batches_x += [x[i:i+n], x[len(y) - i - n: len(y) - i]]
        batches_y += [a, b]
        i += n

    x = np.vstack(batches_x)
    y = np.vstack(batches_y)
    return torch.tensor(x), torch.tensor(y)

train_x, train_y = form_balanced(train_x, train_y, 1000)


In [ ]:
# list(network.parameters())[0].data = torch.Tensor([[10.], [-10]])
# list(network.parameters())[1].data = torch.Tensor([-5., 15])
np.set_printoptions(precision=2)
params = list(network.parameters())
par = []
for p in params:
    par.append(p.detach())
    print (par[-1])

In [ ]:
o1 = train_x @ par[1].T + par[2]
o1

In [ ]:
a1 = torch.sigmoid(o1)
a1

In [ ]:
o2 = a1[:, 0] + a1[:, 1] - 1
mseloss(o2, train_y)

In [ ]:
o2 @ par[2].T

In [ ]:
network.forward(train_x)

In [ ]:
1345639 * 5145

In [ ]:
import pickle
import numpy as np

class A:
    def __init__(self):
        self.v = 1
    def __del__(self):
        print(f"del {self.v}")

a = A()
b = A()
b.v = 2
l = [b]
print("start")
del a
print("after del")
l[0].v
a.v
